In [ ]:
import mysql.connector
from dotenv import load_dotenv
import os

# Carrega variáveis do ficheiro .env
load_dotenv()

# Conexão à base de dados
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)

print("✅ Ligado à base de dados com sucesso.")


In [ ]:
# Célula: Evolução das Vendas por Mês e Ano
query = """
SELECT 
    cm.year_number AS ano,
    cm.calendar_month_number AS mes_num,
    cm.calendar_month_name AS mes_nome,
    SUM(o.sales_order) AS total_vendas
FROM Orders o
JOIN CalendarMonth cm ON o.order_calendar_id = cm.calendar_month_id
GROUP BY cm.year_number, cm.calendar_month_number, cm.calendar_month_name
ORDER BY cm.year_number, cm.calendar_month_number;
"""

df = pd.read_sql(query, conn)
df["ano_mes"] = df["ano"].astype(str) + "-" + df["mes_num"].astype(str).str.zfill(2)

plt.figure(figsize=(14, 6))
sns.lineplot(data=df, x="ano_mes", y="total_vendas", marker="o")
plt.title("Evolução das Vendas por Mês e Ano")
plt.xlabel("Ano-Mês")
plt.ylabel("Total de Vendas")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


O gráfico revela a evolução das vendas ao longo do tempo. Picos ou quedas podem indicar sazonalidades ou efeitos de promoções, ajudando na previsão e planejamento estratégico.

In [ ]:
# Célula: Lucro por Região
query = """
SELECT 
    r.region_name AS regiao,
    SUM(o.profit_order) AS total_lucro
FROM Orders o
JOIN Location l ON o.location_id = l.location_id
JOIN State s ON l.state_id = s.state_id
JOIN Region r ON s.region_id = r.region_id
GROUP BY r.region_name
ORDER BY total_lucro DESC;
"""

df = pd.read_sql(query, conn)

plt.figure(figsize=(10,6))
sns.barplot(data=df, x="regiao", y="total_lucro", palette="viridis")
plt.title("Lucro por Região")
plt.xlabel("Região")
plt.ylabel("Total de Lucro")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Este gráfico identifica quais regiões geram maior lucro, podendo orientar decisões de investimento, marketing e distribuição de recursos.

In [ ]:
# Célula: Top 10 Produtos Mais Vendidos
query = """
SELECT 
    p.product_name,
    SUM(i.quantity) AS quantidade_total
FROM Item i
JOIN Product p ON i.product_id = p.product_id
GROUP BY p.product_name
ORDER BY quantidade_total DESC
LIMIT 10;
"""

df = pd.read_sql(query, conn)

plt.figure(figsize=(10,6))
sns.barplot(data=df, x="quantidade_total", y="product_name", palette="magma")
plt.title("Top 10 Produtos Mais Vendidos")
plt.xlabel("Quantidade Total")
plt.ylabel("Produto")
plt.tight_layout()
plt.show()


Visualiza os produtos com maior volume de vendas, o que pode orientar estratégias de estoque, marketing e análise de demanda.

In [ ]:
# Célula: Vendas por Segmento com ROLLUP
query = """
SELECT 
    IF(GROUPING(c.segment), 'TOTAL', c.segment) AS segmento,
    SUM(o.sales_order) AS total_vendas
FROM Orders o
JOIN Customer c ON o.customer_id = c.customer_id
GROUP BY c.segment WITH ROLLUP;
"""

df = pd.read_sql(query, conn)

plt.figure(figsize=(8,6))
sns.barplot(data=df, x="segmento", y="total_vendas", palette="coolwarm")
plt.title("Vendas por Segmento de Cliente (com Total Geral)")
plt.xlabel("Segmento")
plt.ylabel("Total de Vendas")
plt.tight_layout()
plt.show()


A query com ROLLUP mostra as vendas detalhadas por segmento e inclui uma linha de total geral, facilitando a comparação entre segmentos e a visualização do desempenho global.

In [ ]:
# Célula: Ranking de Produtos por Lucro
query = """
SELECT 
    product_name,
    total_lucro,
    RANK() OVER (ORDER BY total_lucro DESC) AS ranking
FROM (
    SELECT 
        p.product_name,
        SUM(i.profit) AS total_lucro
    FROM Item i
    JOIN Product p ON i.product_id = p.product_id
    GROUP BY p.product_name
) AS sub
ORDER BY ranking
LIMIT 10;
"""

df = pd.read_sql(query, conn)

plt.figure(figsize=(10,6))
sns.barplot(data=df, x="ranking", y="total_lucro", palette="Spectral")
plt.title("Ranking de Produtos por Lucro")
plt.xlabel("Ranking")
plt.ylabel("Total de Lucro")
for index, row in df.iterrows():
    plt.text(row.name, row.total_lucro, row.product_name, color='black', ha="center")
plt.tight_layout()
plt.show()


Este gráfico ordena os produtos por lucro, atribuindo um ranking. Permite identificar rapidamente quais produtos são mais rentáveis.

In [ ]:
# Célula: Acumulação de Vendas por Estado (Running Total)
query = """
SELECT 
    s.state_name,
    c.full_date,
    SUM(o.sales_order) OVER (PARTITION BY s.state_name ORDER BY c.full_date) AS running_total
FROM Orders o
JOIN Location l ON o.location_id = l.location_id
JOIN State s ON l.state_id = s.state_id
JOIN Calendar c ON o.order_calendar_id = c.calendar_id
ORDER BY s.state_name, c.full_date;
"""

df = pd.read_sql(query, conn)
df['full_date'] = pd.to_datetime(df['full_date'])

plt.figure(figsize=(14, 8))
sns.lineplot(data=df, x="full_date", y="running_total", hue="state_name", marker="o")
plt.title("Running Total de Vendas por Estado")
plt.xlabel("Data")
plt.ylabel("Running Total de Vendas")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


O gráfico mostra, para um estado específico, como as vendas vão se acumulando ao longo do tempo. Essa visualização é útil para compreender tendências e a progressão das receitas por estado.

In [ ]:
query7 = """
SELECT 
    sh.ship_mode,
    AVG(DATEDIFF(c2.full_date, c1.full_date)) AS tempo_medio_envio
FROM Orders o
JOIN Shipping sh ON o.shipping_id = sh.shipping_id
JOIN Calendar c1 ON o.order_calendar_id = c1.calendar_id
JOIN Calendar c2 ON o.shipping_calendar_id = c2.calendar_id
GROUP BY sh.ship_mode
ORDER BY tempo_medio_envio;
"""
df7 = pd.read_sql(query7, conn)

plt.figure(figsize=(10,6))
sns.barplot(data=df7, x="ship_mode", y="tempo_medio_envio")
plt.title("Tempo Médio de Envio por Método")
plt.xlabel("Modo de Envio")
plt.ylabel("Tempo Médio (dias)")
plt.tight_layout()
plt.show()


Permite identificar qual o método de envio que apresenta maior ou menor tempo médio entre a encomenda e o envio, ajudando a analisar a eficiência logística.

In [ ]:
query8 = """
SELECT 
    sh.ship_mode,
    ROUND(SUM(o.lost_value_order), 2) AS valor_perdido
FROM Orders o
JOIN Shipping sh ON o.shipping_id = sh.shipping_id
GROUP BY sh.ship_mode
ORDER BY valor_perdido DESC;
"""
df8 = pd.read_sql(query8, conn)

plt.figure(figsize=(10,6))
sns.barplot(data=df8, x="ship_mode", y="valor_perdido")
plt.title("Valor Perdido por Método de Envio")
plt.xlabel("Modo de Envio")
plt.ylabel("Valor Perdido (monetário)")
plt.tight_layout()
plt.show()


Este gráfico mostra o montante de receita perdido devido aos descontos aplicados, agrupado por método de envio – possibilita identificar se há algum canal que afeta significativamente a rentabilidade.

In [ ]:
query9 = """
SELECT 
    cat.category_name,
    SUM(i.sales) AS total_vendas,
    SUM(i.profit) AS total_lucro,
    ROUND(AVG(i.discount), 2) AS desconto_medio
FROM Item i
JOIN Product p ON i.product_id = p.product_id
JOIN Category cat ON p.category_id = cat.category_id
GROUP BY cat.category_name
ORDER BY total_vendas DESC;
"""
df9 = pd.read_sql(query9, conn)

# Gráfico de barras duplo para vendas e lucro
fig, ax1 = plt.subplots(figsize=(10,6))

sns.barplot(data=df9, x="category_name", y="total_vendas", color="skyblue", ax=ax1)
ax1.set_xlabel("Categoria")
ax1.set_ylabel("Total de Vendas", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")
plt.xticks(rotation=45)

# Em cima do mesmo gráfico, adiciona lucro (em outra escala ou sobreposto)
ax2 = ax1.twinx()
sns.lineplot(data=df9, x="category_name", y="total_lucro", marker="o", color="red", ax=ax2)
ax2.set_ylabel("Total de Lucro", color="red")
ax2.tick_params(axis="y", labelcolor="red")

plt.title("Vendas e Lucro por Categoria de Produto")
plt.tight_layout()
plt.show()


A análise combina duas medidas: o volume de vendas e o lucro por categoria, além de apresentar o desconto médio (que pode ser mostrado em um comentário ou tabela extra). Isso ajuda a identificar quais categorias são mais rentáveis e como as políticas de desconto impactam o desempenho.

In [ ]:
query10 = """
SELECT 
    s.state_name,
    SUM(o.sales_order) AS total_vendas,
    SUM(SUM(o.sales_order)) OVER (ORDER BY s.state_name) AS vendas_acumuladas,
    ROUND(SUM(SUM(o.sales_order)) OVER (ORDER BY s.state_name) / 
          (SELECT SUM(sales_order) FROM Orders) * 100, 2) AS percentual_acumulado
FROM Orders o
JOIN Location l ON o.location_id = l.location_id
JOIN State s ON l.state_id = s.state_id
GROUP BY s.state_name
ORDER BY percentual_acumulado;
"""
df10 = pd.read_sql(query10, conn)

plt.figure(figsize=(12,6))
sns.barplot(data=df10, x="state_name", y="percentual_acumulado")
plt.title("Percentual Acumulado de Vendas por Estado")
plt.xlabel("Estado")
plt.ylabel("Percentual Acumulado (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


O gráfico indica, em ordem crescente de percentual acumulado, como as vendas se distribuem entre os estados, permitindo identificar quais estados concentram a maior parcela das receitas.